In [1]:
print("hello")

hello


In [1]:
from pyspark.sql import SparkSession
from delta.tables import DeltaTable

spark = (
    SparkSession.builder
    .appName("spark-health-check")
    .enableHiveSupport()
    .getOrCreate()
)

print("Spark version:", spark.version)
print("Spark master:", spark.sparkContext.master)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/27 20:41:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark version: 3.5.1
Spark master: local[*]


In [8]:
dt = DeltaTable.forName(spark, "users")

In [15]:
dt.toDF().printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- age: string (nullable = true)



In [18]:
dt.update(
    condition= "id='1'",
    set = {
        "name": f"'gowda'"
    }
)

In [2]:
spark.read.csv("./test.csv").show()

+---+-----+---+
|_c0|  _c1|_c2|
+---+-----+---+
| id| name|age|
|  1| prem| 28|
|  2|kumar| 28|
+---+-----+---+



In [3]:
spark.read.csv("./test.csv").write.format('delta').save("/data/managed/test2/")

26/01/27 20:18:06 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

In [7]:
spark.read.load("/data/managed/test2/", format="delta").show()

+---+-----+---+
|_c0|  _c1|_c2|
+---+-----+---+
| id| name|age|
|  1| prem| 28|
|  2|kumar| 28|
+---+-----+---+



In [7]:
spark.read.option("header", True).option("overwriteSchema", "true").csv("./test.csv").write.format("delta").mode('overwrite').saveAsTable("users")

26/01/27 20:43:35 WARN HiveExternalCatalog: Couldn't find corresponding Hive SerDe for data source provider delta. Persisting data source table `spark_catalog`.`default`.`users` into Hive metastore in Spark SQL specific format, which is NOT compatible with Hive.
26/01/27 20:43:35 WARN SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
26/01/27 20:43:36 WARN HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
26/01/27 20:43:36 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
26/01/27 20:43:36 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist


In [6]:
spark.sql("DROP TABLE users").show()

26/01/27 20:43:16 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


++
||
++
++



In [19]:
spark.read.table("users").show()

+---+-----+---+
| id| name|age|
+---+-----+---+
|  1|gowda| 28|
|  2|kumar| 28|
+---+-----+---+

